**Table 4: FF Regression**

For this code, datasets required:
- CSV: market_cap, FF3, FF3Y, FF5, FFMOM, HXZ, at_HML(Table2 output), at_df_yret(Table2 output), portfolio_at2(Table2 output)
- XLSX: rf6023

In [ ]:
import pandas as pd
import statsmodels.api as sm
import numpy as np

In [ ]:
# merge yret data with at_df
portfolio_at = pd.read_csv('portfolio_at2.csv')
market_cap=pd.read_csv('market_cap.csv') # monthly mktcap for each gvkey, with year and month num
rf = pd.read_excel('rf6023.xlsx')
print(rf.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   year    64 non-null     int64  
 1   rf      64 non-null     float64
dtypes: float64(1), int64(1)
memory usage: 1.1 KB
None


In [ ]:
# creat year and month
market_cap['MthCalDt'] = pd.to_datetime(market_cap['MthCalDt'], format='%Y/%m/%d')
market_cap['year'] = market_cap['MthCalDt'].dt.year
market_cap['month'] = market_cap['MthCalDt'].dt.month
market_cap = market_cap.drop(columns=['MthCalDt'])
print(market_cap.describe())
market_cap.head()

             PERMNO        MthCap          gvkey          year         month
count  21692.000000  2.169200e+04   21692.000000  21692.000000  21692.000000
mean   77955.165775  4.584563e+06  153503.101835   2011.761110      6.515029
std    27012.647796  9.662385e+06   32019.133455      6.162657      3.457175
min    12009.000000  3.681480e+03  105365.000000   1996.000000      1.000000
25%    86316.000000  4.906814e+05  127575.000000   2007.000000      4.000000
50%    88917.000000  1.467253e+06  155394.000000   2012.000000      7.000000
75%    90870.000000  4.103633e+06  175404.000000   2017.000000     10.000000
max    93387.000000  1.332086e+08  315318.000000   2023.000000     12.000000


,PERMNO,MthCap,gvkey,year,month
0,86111,376581.19,105365,1998,6
1,86111,2269944.00,105365,1998,7
2,86111,1489650.75,105365,1998,8
3,86111,2411815.50,105365,1998,9
4,86111,2068959.38,105365,1998,10


In [ ]:
def ffreg(target,features,datay,dataX): # target is 'string input', features is list of 'string variables'
  data = dataX.merge(datay,on=['year','month'],how='inner')
  X = sm.add_constant(data[features]) # add constant term for alpha estimation
  y = data[target]
  model = sm.OLS(y, X).fit()
  print(model.summary())

In [ ]:
# create data with monthly return and group tag
market_cap['mret'] = (market_cap['MthCap'] / market_cap['MthCap'].shift(1) - 1)*100
market_cap = market_cap.dropna(subset=['mret'])
mret = market_cap[['gvkey','year','month','mret','MthCap']]
mret = pd.merge(portfolio_at[['fyear','gvkey','group_tag']],mret,left_on=['gvkey','fyear'],right_on=['gvkey','year'], how='inner')
mret = mret.merge(rf, on='year',how='inner')
mret = mret.drop('fyear',axis=1)
mret.head()

,gvkey,group_tag,year,month,mret,MthCap,rf
0,105365,H,1998,7,502.776788,2269944.00,4.33
1,105365,H,1998,8,-34.375000,1489650.75,4.33
2,105365,H,1998,9,61.904762,2411815.50,4.33
3,105365,H,1998,10,-14.215686,2068959.38,4.33
4,105365,H,1998,11,7.499271,2224116.25,4.33


In [ ]:
# datay: H-L month return
H = mret[mret['group_tag']=='H']
L = mret[mret['group_tag']=='L']

# Filter the original DataFrame to include only rows where tag is 'H'
m_ret_H = pd.DataFrame(columns = ['year','month','mret_H'])
for year in H['year'].unique():
    groupret_m_H = {}
    year_H = H[H['year'] == year].copy()
    for month in year_H['month'].unique():
      month_H = year_H[year_H['month'] == month].copy()
      month_H['weight'] = month_H['MthCap'] / month_H['MthCap'].sum()
      m_H_ret = (month_H['mret'] * month_H['weight']).sum()
      groupret_m_H[month] = m_H_ret
    groupret_m_H = pd.DataFrame(list(groupret_m_H.items()), columns=['month', 'mret_H'])
    groupret_m_H['year'] = year*np.ones(len(groupret_m_H)).astype(int)
    m_ret_H = pd.concat([m_ret_H, groupret_m_H], ignore_index=True)

# Filter the original DataFrame to include only rows where tag is 'L'
m_ret_L = pd.DataFrame(columns = ['year','month','mret_L'])
for year in L['year'].unique():
    groupret_m_L = {}
    year_L = L[L['year'] == year].copy()
    for month in year_L['month'].unique():
      month_L = year_L[year_L['month'] == month].copy()
      month_L['weight'] = month_L['MthCap'] / month_L['MthCap'].sum()
      m_L_ret = (month_L['mret'] * month_L['weight']).sum()
      groupret_m_L[month] = m_L_ret
    groupret_m_L = pd.DataFrame(list(groupret_m_L.items()), columns=['month', 'mret_L'])
    groupret_m_L['year'] = year * np.ones(len(groupret_m_L)).astype(int)
    m_ret_L = pd.concat([m_ret_L, groupret_m_L], ignore_index=True)

H_L = pd.merge(m_ret_H, m_ret_L, on=['year','month'], how='inner')
H_L['mret'] = H_L['mret_H']- H_L['mret_L'] # for regression convenience, we name H-L ret as 'ret_prem'
print(H_L)

     year month      mret_H     mret_L        mret
0    1998     7  181.184974   7.513312  173.671661
1    1998     8  -15.875810 -18.971726    3.095917
2    1998     9   25.613813 -43.882740   69.496553
3    1998    10   -6.259527  17.096958  -23.356485
4    1998    11    8.660271  19.268675  -10.608403
..    ...   ...         ...        ...         ...
292  2022     8   -5.695190   1.341448   -7.036638
293  2022     9  -10.727473  -5.440631   -5.286842
294  2022    10    4.433824  15.705974  -11.272150
295  2022    11   10.812030   2.487602    8.324428
296  2022    12   -9.486876  -3.018951   -6.467924

[297 rows x 5 columns]


In [ ]:
#split target in groups
tagl = mret[mret['group_tag']== 'L']
tag2 = mret[mret['group_tag']== '2']
tag3 = mret[mret['group_tag']== '3']
tag4 = mret[mret['group_tag']== '4']
tagh = mret[mret['group_tag']== 'H']
taghl = H_L
datay =[tagl,tag2,tag3,tag4,tagh,taghl]
taghl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297 entries, 0 to 296
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   year    297 non-null    object 
 1   month   297 non-null    object 
 2   mret_H  297 non-null    float64
 3   mret_L  297 non-null    float64
 4   mret    297 non-null    float64
dtypes: float64(3), object(2)
memory usage: 11.7+ KB


In [ ]:
# load features
ff3 = pd.read_csv('FF3.csv').dropna()
ffmom = pd.read_csv('FFMOM.csv').dropna()
ff5 = pd.read_csv('FF5.csv').dropna()
print(ff3.info())
print(ffmom.info())
print(ff5.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1173 entries, 0 to 1172
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    1173 non-null   int64  
 1   MKT     1173 non-null   float64
 2   SMB     1173 non-null   float64
 3   HML     1173 non-null   float64
 4   rf      1173 non-null   float64
dtypes: float64(4), int64(1)
memory usage: 45.9 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    1167 non-null   int64  
 1   MOM     1167 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 18.4 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 729 entries, 0 to 728
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    729 non-null    int64  
 1   MKT     729 non-null    float64
 2   SMB     729 

In [ ]:
# seperate date in feature data into 'year' and 'month'
# Extract first 4 digits of 'date' for year, and last 2 digits for month
dataX = [ff3,ffmom,ff5]
for i in dataX:
  i['date'] = i['date'].astype(int)
  i['year'] = i['date'] // 100
  i['month'] = i['date'] % 100
  print(i.head())

ff3 = ff3.drop(columns=['date'])
ffmom = ffmom.drop(columns=['date'])
ff5 = ff5.drop(columns=['date'])
# merge MOM with FF3
ffmom_merge = ff3.merge(ffmom,on=['year','month'],how='inner')
ffmom_merge

     date   MKT   SMB   HML    rf  year  month
0  192607  2.96 -2.56 -2.43  0.22  1926      7
1  192608  2.64 -1.17  3.82  0.25  1926      8
2  192609  0.36 -1.40  0.13  0.23  1926      9
3  192610 -3.24 -0.09  0.70  0.32  1926     10
4  192611  2.53 -0.10 -0.51  0.31  1926     11
     date   MOM  year  month
0  192701  0.36  1927      1
1  192702 -2.14  1927      2
2  192703  3.61  1927      3
3  192704  4.30  1927      4
4  192705  3.00  1927      5
     date   MKT   SMB   HML   RMW   CMA    rf  year  month
0  196307 -0.39 -0.41 -0.97  0.68 -1.18  0.27  1963      7
1  196308  5.07 -0.80  1.80  0.36 -0.35  0.25  1963      8
2  196309 -1.57 -0.52  0.13 -0.71  0.29  0.27  1963      9
3  196310  2.53 -1.39 -0.10  2.80 -2.01  0.29  1963     10
4  196311 -0.85 -0.88  1.75 -0.51  2.24  0.27  1963     11


,MKT,SMB,HML,rf,year,month,MOM
0,-0.06,-0.37,4.54,0.25,1927,1,0.36
1,4.18,0.04,2.94,0.26,1927,2,-2.14
2,0.13,-1.65,-2.61,0.30,1927,3,3.61
3,0.46,0.30,0.81,0.25,1927,4,4.30
4,5.44,1.53,4.73,0.30,1927,5,3.00
...,...,...,...,...,...,...,...
1162,8.84,-0.02,1.64,0.44,2023,11,2.75
1163,4.87,6.34,4.93,0.43,2023,12,-5.51
1164,0.71,-5.09,-2.38,0.47,2024,1,5.19
1165,5.06,-0.24,-3.49,0.42,2024,2,4.96


In [ ]:
# seperate regression for each group
# regression function: ffreg(target,features,datay,dataX)
y = 'mret'
tag = mret['group_tag'].unique().tolist()
tag.append('H-L')

# Panel A: CAPM MKT
id=0
for i in datay:
  print(f'for CAPM Model for target {tag[id]}: \n')
  id+=1
  ffreg(y,['MKT'],i,ff3)

# Panel B: FF3 MKT/SMB/HML
id=0
for i in datay:
  print(f'for FF3 Model for target {tag[id]}: \n')
  id+=1
  ffreg(y,['MKT','SMB','HML'],i,ff3)

# Panel C: FF4 MKT/SMB/HML/MOM
id=0
for i in datay:
  print(f'for FF4 Model for target {tag[id]}: \n')
  id+=1
  ffreg(y,['MKT','SMB','HML','MOM'],i,ffmom_merge)

# Panel D: FF5 MKT/SMB/HML/RMW/CMA
id=0
for i in datay:
  print(f'for FF5 Model for target {tag[id]}: \n')
  id+=1
  ffreg(y,['MKT','SMB','HML','RMW','CMA'],i,ff5)

for CAPM Model for target H: 

                            OLS Regression Results                            
Dep. Variable:                   mret   R-squared:                       0.035
Model:                            OLS   Adj. R-squared:                  0.035
Method:                 Least Squares   F-statistic:                     115.4
Date:                Sat, 18 May 2024   Prob (F-statistic):           1.84e-26
Time:                        18:20:55   Log-Likelihood:                -15299.
No. Observations:                3190   AIC:                         3.060e+04
Df Residuals:                    3188   BIC:                         3.061e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.3328

In [ ]:
# Panel E: HXZ MKT/SMB/IA/ROE -> annual data(the other models use monthly data)
hxz = pd.read_csv('HXZ.csv').dropna()
hxz.drop_duplicates(inplace=True)
hxz['roe'] = hxz['roe']*100

In [ ]:
# merge market cap data to weight I/A
hxz_ia = pd.DataFrame(columns=['gvkey', 'roe', 'year', 'at', 'del_at', 'IA'])
for gvkey in hxz['gvkey'].unique().tolist():
  key_data = hxz[hxz['gvkey'] == gvkey].copy()
  key_data = key_data.sort_values(by='year')
  key_data['del_at'] = key_data['at'].diff() # at_t - at_t-1
  key_data['IA'] = key_data['del_at']/key_data['at'].shift(1)*100
  hxz_ia = pd.concat([hxz_ia, key_data], ignore_index=True)
hxz_ia

,gvkey,roe,year,at,del_at,IA
0,4819,6.700000,1991,89.423,NaN,NaN
1,4819,-22.325000,1992,89.871,0.448,0.500990
2,4819,8.150000,1993,97.837,7.966,8.863816
3,4819,10.550000,1994,101.685,3.848,3.933072
4,4819,12.425000,1995,102.053,0.368,0.361902
...,...,...,...,...,...,...
13782,4839,-3.800000,2019,114047.000,-602.000,-0.525081
13783,4839,6.925000,2020,116744.000,2697.000,2.364815
13784,4839,31.181818,2021,108996.000,-7748.000,-6.636744
13785,179597,3.200000,2019,152.757,NaN,NaN


In [ ]:
# merge market cap data to weight I/A
hxz_merge = pd.merge(market_cap[['gvkey','year','MthCap']], hxz_ia, on=['year','gvkey'], how='inner')
IA={}
ROE={}
for year in hxz_merge['year'].unique():
  year_data = hxz_merge[hxz_merge['year'] == year].copy()
  sum = year_data['MthCap'].sum()
  year_data['weights'] = year_data['MthCap']/sum
  IA[year] = (year_data['weights']*year_data['IA']).sum()
  ROE[year] = (year_data['weights']*year_data['roe']).sum()

IA = pd.DataFrame(list(IA.items()), columns=['year', 'IA'])
ROE = pd.DataFrame(list(ROE.items()), columns=['year', 'ROE'])

In [ ]:
# dataX preparation: merge I/A, ROE, MKT, SMB
at_df_yret = pd.read_csv('at_df_yret.csv')
H_L = pd.read_csv('at_HML.csv')
H_L.rename(columns={'H-L': 'yret','year':'fyear'}, inplace=True) # for regression convenience
ff3y = pd.read_csv('FF3Y.csv')
HXZ = IA.merge(ROE,on='year',how='inner')
HXZ = HXZ.merge(ff3y[['year','MKT','SMB']], on='year', how='inner')
# datay preparation: split by groups
tagl = at_df_yret[at_df_yret['group_tag']== 'L']
tag2 = at_df_yret[at_df_yret['group_tag']== '2']
tag3 = at_df_yret[at_df_yret['group_tag']== '3']
tag4 = at_df_yret[at_df_yret['group_tag']== '4']
tagh = at_df_yret[at_df_yret['group_tag']== 'H']
taghl = H_L
datayy =[tagl,tag2,tag3,tag4,tagh,taghl]
tagh.info()

<class 'pandas.core.frame.DataFrame'>
Index: 323 entries, 0 to 1433
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   fyear        323 non-null    int64  
 1   gvkey        323 non-null    int64  
 2   at           323 non-null    float64
 3   p_at         323 non-null    float64
 4   group_tag    323 non-null    object 
 5   trade        323 non-null    int64  
 6   weight_l     323 non-null    float64
 7   weight_s     323 non-null    float64
 8   weight       323 non-null    float64
 9   yret         323 non-null    float64
 10  cap          323 non-null    float64
 11  rf           323 non-null    float64
 12  Implied ERP  323 non-null    float64
 13  ret_prem     323 non-null    float64
dtypes: float64(10), int64(3), object(1)
memory usage: 37.9+ KB


In [ ]:
# Panle E Regression
tag = at_df_yret['group_tag'].unique().tolist()
tag.append('H-L')

id=0
for i in datayy:
  print(f'for HXZ Model for target {tag[id]}: \n')
  id+=1
  data = HXZ.merge(i,left_on=['year'],right_on=['fyear'],how='inner')
  X = sm.add_constant(data[['MKT','SMB','IA','ROE']]) # add constant term for alpha estimation
  y = data['yret']
  model = sm.OLS(y, X).fit()
  print(model.summary())

for HXZ Model for target H: 

                            OLS Regression Results                            
Dep. Variable:                   yret   R-squared:                       0.070
Model:                            OLS   Adj. R-squared:                  0.056
Method:                 Least Squares   F-statistic:                     5.075
Date:                Sat, 18 May 2024   Prob (F-statistic):           0.000587
Time:                        18:24:16   Log-Likelihood:                -1535.7
No. Observations:                 276   AIC:                             3081.
Df Residuals:                     271   BIC:                             3099.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         16.3973 